In [1]:
from src.models import *
from src.gfo import GFOProblem, SOCallback, blocker, build_rand_blocks, get_model_params, set_model_state

# Train on MNIST
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm

import numpy as np
import pandas as pd
from sklearn.metrics import (
            f1_score,
        )
import matplotlib.pyplot as plt

In [2]:
# Load MNIST
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)
trainset = torchvision.datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
testset = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
train_loader = DataLoader(trainset, batch_size=128, shuffle=True)
test_loader = DataLoader(testset, batch_size=10000, shuffle=False)

In [3]:
def test(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    f1 = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum().item()
            f1 += f1_score(target.view_as(pred).cpu().numpy(), pred.cpu().numpy(), average='macro')


    return total_loss / len(test_loader), correct / len(test_loader.dataset), f1 / len(test_loader)

In [7]:
from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.optimize import minimize

# Parameter Setting
NP = 100
block_size = 100
# Define model
model = LeNet(in_channels=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

init_params = get_model_params(model)
D = len(init_params)
print(f"Original dims: {D} D")

import os
import pickle 

codebook = {}
if os.path.exists(f'out/codebook_D{D}_blocksize{block_size}.pkl'):
    with open(f'out/codebook_D{D}_blocksize{block_size}.pkl', 'rb') as f:
        codebook = pickle.load(f)
else:
    codebook = build_rand_blocks(D, block_size=block_size)
    
    with open(f'out/codebook_D{D}_blocksize{block_size}.pkl', 'wb') as f:
        pickle.dump(codebook, f)

bD = len(codebook)
print(f"Blocked dims: {bD} D")
x0 = np.random.uniform(low=-5, high=5, size=(bD))
init_params = blocker(init_params, codebook)
# x0 = init_params.copy()
# init_pop = np.random.normal(loc=init_params, scale=0.1, size=(NP, bD))

# random_indices = np.random.choice(
#             np.arange(len(trainset)), size=1024, replace=False
#         )
# random_dataset = Subset(trainset, random_indices)
# data_loader = DataLoader(random_dataset, batch_size=128, shuffle=True)
data_loader = DataLoader(trainset, batch_size=2048, shuffle=True)
problem = GFOProblem(n_var=bD, model=model, dataset=trainset, test_loader=test_loader, train_loader=data_loader, 
                      set_model_state=set_model_state,
                      batch_size=1024, device=device, criterion="f1",
                      block=True, codebook=codebook, orig_dims=D)
out={"F": []}
problem._evaluate(np.array([x0, init_params]), out=out)
print(out)

algorithm = CMAES(
    x0=x0,
    sigma=0.01,
    tolconditioncov=0,
    tolfacupx=np.inf,
    tolupsigma=np.inf,
    tolfun=0,
    tolfunhist=0,
    tolstagnation=np.inf,
    tolx=0,
    # restarts=4, 
    # bipop=True,
)

csv_path = f"out/LeNet_block_bs{block_size}_gfo_f1_2048data_cmaes_hist.csv"
plt_path = f"out/LeNet_block_bs{block_size}_gfo_f1_2048data_cmaes_plt.pdf"
df = pd.DataFrame({
            'n_step': [0],
            'n_eval': [1],
            'f_best': [out["F"][0]],
            'f_avg': [out["F"][0]],
            'f_std': [0],
            'test_f1_best': problem.test_func(x0),
        })
df.to_csv(csv_path, index=False)

res = minimize(problem,
               algorithm,
               ('n_eval', 1000000),
               callback=SOCallback(k_steps=100, csv_path=csv_path, plt_path=plt_path),
               seed=None,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Original dims: 61706 D
Blocked dims: 618 D
{'F': array([0.97722662, 0.98028169])}
n_gen  |  n_eval  |     f_avg     |     f_min     |     f_gap     |     sigma     | min_std  | max_std  |   axis  
     1 |        1 |  0.9767754320 |  0.9767754320 |  0.9767754320 |  0.0100000000 |  0.01000 |  0.01000 |  1.00005
     2 |       24 |  0.9747181723 |  0.9663996138 |  0.9663996138 |  0.0098859124 |  0.00989 |  0.00989 |  1.00005
     3 |       47 |  0.9715038988 |  0.9628229029 |  0.9628229029 |  0.0097829318 |  0.00978 |  0.00978 |  1.00005
     4 |       70 |  0.9672761249 |  0.9582091607 |  0.9582091607 |  0.0096884933 |  0.00969 |  0.00969 |  1.00005
     5 |       93 |  0.9616082866 |  0.9419231415 |  0.9419231415 |  0.0096010531 |  0.00960 |  0.00960 |  1.00005
     6 |      116 |  0.9561727784 |  0.9419231415 |  0.9419231415 |  0.0095177178 |  0.00952 |  0.00952 |  1.00005
     7 |      139 |  0.9493108774 |  0.9295005798 |  0.9295005798 |  0.0094400808 |  0.00944 |  0.00944 |  1.0042

KeyboardInterrupt: 